In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import cv2
import pandas as pd


from render import Renderer
# Renderer is a class(package) which helps to plot in cv2
class object(Renderer):
    def __init__(self,data,recordLocation = None):
        super().__init__(recordLocation=recordLocation)
        # self.its = 0
        self.q1 = 0     # q1 is the angle made by the first limb of manipulator with x axis
        self.q2 = 0     # q2 is the angle made by the second limb of manipulator with x axis
        self.theta = 0  # theta is the angle made by the limb 2 w.r.t to limb 1
        self.l1 = 200   # length of the first limb
        self.l2 = 380   # length of the second limb
        self.points = set()
        self.data = data
    def getInfo(self):
        info = {
            'Fx':round(self.Fx,4),
            'Fy':round(self.Fy,4), 
            'torque1': round(self.torque1,4),
            'torque2': round(self.torque2,4),
        }
        return info
    def plot(self):

        pass
    def inner(self,x,y):
        # Fx and Fy are the x and y components of the force that is applied at the given point on the wall 
        self.Fx = self.data[1,0]
        self.Fy = self.data[1,1]
        
        self.theta=(math.acos(((x)**2+(y)**2-(self.l1)**2-(self.l2)**2)/(2*self.l1*self.l2))) 
        
        self.q1 = math.atan((y)/(x))- math.atan((self.l2*math.sin(self.theta))/(self.l1+(self.l2*math.cos(self.theta))))
        self.q2 = self.q1+self.theta
        
        
        self.torque1 = (self.Fy*self.l1*math.cos(self.q1)-self.Fx*self.l1*math.sin(self.q1))/1000 # torque applied at the joint 1
        self.torque2 = (self.Fy*self.l2*math.cos(self.q2)-self.Fx*self.l2*math.sin(self.q2))/1000 # torque applied at the joint 2
        
        # (x1,y1) is the coordinate of the joint 1
        self.x1 = self.l1*math.cos(self.q1)
        self.y1 = self.l1*math.sin(self.q1)
        
        # (x2,y2) is the coordinate of the joint 2
        self.x2 = self.x1+self.l2*math.cos(self.q2)
        self.y2 = self.y1+self.l2*math.sin(self.q2)
        anim.render(height=600,pause=100)
        
    def step(self,dt):
        # (x,y) denotes the initial position of end effector
        self.x0 = 100
        self.y0 = 400
        self.inner(self.x0,self.y0)
        # (x,y) denotes the position of point on the wall at which force is applied
        self.x = self.data[0,0]  
        self.y = self.data[0,1]
        self.k1 = (self.x-self.x0)/(100*dt)
        self.k2 = (self.y-self.y0)/(100*dt)
        for i in range(0,100):
            self.x0 += self.k1*dt
            self.y0 += self.k2*dt
            self.inner(self.x0,self.y0)
        
    def draw(self,image):
        cv2.line(image,(0,0),(int(self.x1),int(self.y1)),(0,255,0),2)  # limb 1
        cv2.line(image,(int(self.x1),int(self.y1)),(int(self.x2),int(self.y2)),(0,0,255),2) # limb 2
        cv2.circle(image,(int(self.x2),int(self.y2)),1,(255,0,0),-1)  # trajectory traced by our manipulator
        return image 
    
user = list(map(int,input(" Enter the position of a point on wall(x,y) and the force component(Fx,Fy) that manipulator applied on the wall;(like: 200 400 40 50): ").split()))
data = np.array(user) # converting the list into array
data = data.reshape(2,2)  # reshaping to 2X2 matrix  
anim = object(data,recordLocation ='anim.mp4')
anim.step(1)


        

 Enter the position of a point on wall(x,y) and the force component(Fx,Fy) that manipulator applied on the wall;(like: 200 400 40 50): 200 400 40 50
